<a href="https://colab.research.google.com/github/PierfrancescoElia/AICyberLabs/blob/main/Lab03/lab03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3: Natural Language Processing (NLP)

| Member | ID |
| -- | -- |
| Alessandro Meneghini| s332228 |
| Pierfrancesco Elia | s331497 |
| Ankesh Porwal | s328746 |


In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
import time


Task 1

In [4]:
# Set random seed for reproducibility
np.random.seed(42)

# import the dataset
dataset_url = 'https://github.com/PierfrancescoElia/AICyberLabs/raw/refs/heads/main/Lab03/cyberlab.bigcsv'
df = pd.read_csv(dataset_url)

# import train and test json
train_url = 'https://raw.githubusercontent.com/PierfrancescoElia/AICyberLabs/refs/heads/main/Lab03/train.json'
test_url = 'https://raw.githubusercontent.com/PierfrancescoElia/AICyberLabs/refs/heads/main/Lab03/test.json'

df

,session,timestamps_statements,country_name
0,enable ; system ; shell ; sh ; cat /proc/mount...,2019-09-01 00:00:10.493808+00:00,Israel
1,enable ; system ; shell ; sh ; cat /proc/mount...,2019-09-01 00:38:41.134935+00:00,Israel
2,enable ; system ; shell ; sh ; cat /proc/mount...,2019-09-01 00:39:26.263383+00:00,Israel
3,enable ; system ; shell ; sh ; cat /proc/mount...,2019-09-01 00:40:45.132152+00:00,Israel
4,enable ; system ; shell ; sh ; cat /proc/mount...,2019-09-01 00:54:51.783935+00:00,Israel
...,...,...,...
174257,cat /proc/cpuinfo | grep name | wc -l ; echo -...,2019-12-30 23:37:10.487881+00:00,Thailand
174258,cat /proc/cpuinfo | grep name | wc -l ; echo -...,2019-12-30 23:37:10.923944+00:00,Thailand
174259,cat /proc/cpuinfo | grep name | wc -l ; echo -...,2019-12-30 23:41:46.601903+00:00,Netherlands
174260,cat /proc/cpuinfo | grep name | wc -l ; echo -...,2019-12-30 23:43:34.981985+00:00,Colombia
